In [1]:
import json
import requests
import pandas as pd 
import numpy as np

### Read the disease and  symptom name of mesh

In [2]:
dis = pd.read_csv("../final/cls_siamese/Mesh_files/Mesh_disease_unique.csv",index_col=0)
print(dis.shape)

dis.head()

(4536, 2)


,Mesh_dis_name,Mesh_id
0,Infections,C01
1,"Aneurysm, Infected",C01.069
2,"Arthritis, Infectious",C01.100
3,"Arthritis, Reactive",C01.100.500
4,Asymptomatic Infections,C01.125


In [63]:
sym = pd.read_csv("../final/cls_siamese/Mesh_files/Mesh_symptom.csv",index_col=0)
print(sym.shape)

sym.head()

(385, 2)


,Mesh_sym_name,Mesh_id
0,"Aging, Premature",C23.888.069
1,Asthenia,C23.888.089
2,Body Temperature Changes,C23.888.119
3,Fever,C23.888.119.344
4,Fever of Unknown Origin,C23.888.119.344.345


### Esearch

- Solutions for ID sets exceeding 100000：
    - First, use esearch to retrieve the GI numbers of these sequences and publish them to the history server，
    - Then use multiple efetch calls to retrieve data in a batch of 100000.

In [1]:
# extract pubmed data
def get_page(url):
    #requests.get 自带 json.load
    page = requests.get(url)
    page = page.json()
    return  page

### EFetch

In [5]:
# return:DataFrame
# string:String to split
# term:Query term as column name of DF
def strTocsv(string):
    
    strlist = string.split("\n")
    df = pd.DataFrame(strlist)
    
    return df

def strTolist(string):
    strlist = string.split("\n")
    
    return strlist

### Query diseases

In [6]:
#According to query words，return dataframe of PMID list

def queryDisByMesh(dis_term):

    #assemble the esearch URL
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&mindate=1966&maxdate=2020/12&retmode=json&usehistory=y"
    # query_term = "&term=Breast+Neoplasms[MAJR]"
    
    query_term = "&term={}[MAJR]".format(dis_term.replace(" ","+"))
    
    url = base_url+query_term
    #post the esearch URL
    esearch_output = get_page(url)
    
    #parse WebEnv, QueryKey and Count (# records retrieved)
    web = esearch_output["esearchresult"]["webenv"]
    key = esearch_output["esearchresult"]["querykey"]
    count = int(esearch_output["esearchresult"]["count"])
    
    #retrieve data in batches of 100000(最大值)
    retmax = 100000
    retstart =0
    # result = pd.DataFrame()
    result = []
    
    while retstart < count:
        efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&WebEnv={}".format(web)
        efetch_url = efetch_url + "&query_key={}&retstart={}".format(key,retstart)
        efetch_url = efetch_url + "&retmax={}&rettype=uilist&retmode=text".format(retmax)
        
        efetch_out = requests.get(efetch_url).text #str
        idlist = strTolist(efetch_out)[:-1]  # 列表最后一个元素为空，去掉
        result.extend(idlist)
    
        retstart+=retmax
        
    result_df = pd.DataFrame({dis_term:result})
    
    print(result_df.shape)
    print("长度是否相等：",result_df.shape[0]==count)
    
    return result_df

In [10]:
for disease in dis["Mesh_dis_name"].iloc[4455:]:
    result = queryDisByMesh(disease)
    result.to_csv("dis_PMIDlist/{}.csv".format(disease))

(420, 1)
长度是否相等： True
(2063, 1)
长度是否相等： True
(36757, 1)
长度是否相等： True
(1978, 1)
长度是否相等： True
(7454, 1)
长度是否相等： True
(236, 1)
长度是否相等： True
(4638, 1)
长度是否相等： True
(1670, 1)
长度是否相等： True
(139789, 1)
长度是否相等： True
(1402, 1)
长度是否相等： True
(190, 1)
长度是否相等： True
(31057, 1)
长度是否相等： True
(19568, 1)
长度是否相等： True
(6464, 1)
长度是否相等： True
(2232, 1)
长度是否相等： True
(1729, 1)
长度是否相等： True
(1981, 1)
长度是否相等： True
(1187, 1)
长度是否相等： True
(134, 1)
长度是否相等： True
(3641, 1)
长度是否相等： True
(4138, 1)
长度是否相等： True
(2703, 1)
长度是否相等： True
(6728, 1)
长度是否相等： True
(2815, 1)
长度是否相等： True
(1130, 1)
长度是否相等： True
(5040, 1)
长度是否相等： True
(980, 1)
长度是否相等： True
(1842, 1)
长度是否相等： True
(2989, 1)
长度是否相等： True
(132, 1)
长度是否相等： True
(11835, 1)
长度是否相等： True
(511, 1)
长度是否相等： True
(13929, 1)
长度是否相等： True
(7171, 1)
长度是否相等： True
(4645, 1)
长度是否相等： True
(1036, 1)
长度是否相等： True
(1109, 1)
长度是否相等： True
(51, 1)
长度是否相等： True
(24108, 1)
长度是否相等： True
(1953, 1)
长度是否相等： True
(76984, 1)
长度是否相等： True
(7737, 1)
长度是否相等： True
(3109, 1)
长度是否相等： True
(19721, 1)


In [13]:
print(disease)
print("疾病数：",len(dis["Mesh_dis_name"]))
dis["Mesh_dis_name"].iloc[4536]

Needlestick Injuries
疾病数： 4536


IndexError: single positional indexer is out-of-bounds

### Query symptoms

In [17]:
#According to query words，return dataframe of PMID list

def querySymByMesh(sym_term):

    #assemble the esearch URL
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&mindate=1966&maxdate=2020/12&retmode=json&usehistory=y"
    # query_term = "&term=Breast+Neoplasms[MAJR]"
    
    query_term = "&term={}[MH]".format(sym_term.replace(" ","+"))
    
    url = base_url+query_term
    #post the esearch URL
    esearch_output = get_page(url)
    
    #parse WebEnv, QueryKey and Count (# records retrieved)
    web = esearch_output["esearchresult"]["webenv"]
    key = esearch_output["esearchresult"]["querykey"]
    count = int(esearch_output["esearchresult"]["count"])
    
    #retrieve data in batches of 100000(max_value)
    retmax = 100000
    retstart =0
    # result = pd.DataFrame()
    result = []
    
    while retstart < count:
        efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&WebEnv={}".format(web)
        efetch_url = efetch_url + "&query_key={}&retstart={}".format(key,retstart)
        efetch_url = efetch_url + "&retmax={}&rettype=uilist&retmode=text".format(retmax)
        
        efetch_out = requests.get(efetch_url).text #str
        idlist = strTolist(efetch_out)[:-1]  # 列表最后一个元素为空，去掉
        result.extend(idlist)
    
        retstart+=retmax
        
    result_df = pd.DataFrame({sym_term:result})
    
    print(result_df.shape)
    print("长度是否相等：",result_df.shape[0]==count)
    
    return result_df

In [19]:
lenthDict ={}
for symptom in sym["Mesh_sym_name"]:
    result = querySymByMesh(symptom)
    result.to_csv("sym_PMIDlist/{}.csv".format(symptom))
    lenthDict[symptom]= len(result)

(1108, 1)
长度是否相等： True
(1388, 1)
长度是否相等： True
(51132, 1)
长度是否相等： True
(41710, 1)
长度是否相等： True
(4304, 1)
长度是否相等： True
(26, 1)
长度是否相等： True
(9806, 1)
长度是否相等： True
(472368, 1)
长度是否相等： True
(41945, 1)
长度是否相等： True
(2798, 1)
长度是否相等： True
(74093, 1)
长度是否相等： True
(148, 1)
长度是否相等： True
(32879, 1)
长度是否相等： True
(670, 1)
长度是否相等： True
(43284, 1)
长度是否相等： True
(5632, 1)
长度是否相等： True
(5185, 1)
长度是否相等： True
(1846, 1)
长度是否相等： True
(228471, 1)
长度是否相等： True
(221653, 1)
长度是否相等： True
(274, 1)
长度是否相等： True
(273, 1)
长度是否相等： True
(21833, 1)
长度是否相等： True
(9948, 1)
长度是否相等： True
(6298, 1)
长度是否相等： True
(206, 1)
长度是否相等： True
(5536, 1)
长度是否相等： True
(237, 1)
长度是否相等： True
(4064, 1)
长度是否相等： True
(41, 1)
长度是否相等： True
(38876, 1)
长度是否相等： True
(652, 1)
长度是否相等： True
(2308, 1)
长度是否相等： True
(4936, 1)
长度是否相等： True
(9079, 1)
长度是否相等： True
(703, 1)
长度是否相等： True
(210, 1)
长度是否相等： True
(2244, 1)
长度是否相等： True
(30750, 1)
长度是否相等： True
(2178, 1)
长度是否相等： True
(59, 1)
长度是否相等： True
(529, 1)
长度是否相等： True
(671, 1)
长度是否相等： True
(3331, 1)
长度是

(362, 1)
长度是否相等： True
(124, 1)
长度是否相等： True
(61616, 1)
长度是否相等： True
(461, 1)
长度是否相等： True
(60, 1)
长度是否相等： True
(1313, 1)
长度是否相等： True
(23603, 1)
长度是否相等： True
(1303, 1)
长度是否相等： True
(9485, 1)
长度是否相等： True
(2303, 1)
长度是否相等： True
(299, 1)
长度是否相等： True
(67, 1)
长度是否相等： True
(276, 1)
长度是否相等： True
(12941, 1)
长度是否相等： True
(1221, 1)
长度是否相等： True
(23877, 1)
长度是否相等： True
(251, 1)
长度是否相等： True
(218, 1)
长度是否相等： True
(3935, 1)
长度是否相等： True
(15213, 1)
长度是否相等： True
(6354, 1)
长度是否相等： True
(4509, 1)
长度是否相等： True
(286, 1)
长度是否相等： True
(220, 1)
长度是否相等： True
(461, 1)
长度是否相等： True
(1275, 1)
长度是否相等： True
(486, 1)
长度是否相等： True
(82632, 1)
长度是否相等： True
(780, 1)
长度是否相等： True
(40950, 1)
长度是否相等： True
(424, 1)
长度是否相等： True
(953, 1)
长度是否相等： True
(490, 1)
长度是否相等： True
(4999, 1)
长度是否相等： True
(118, 1)
长度是否相等： True
(32781, 1)
长度是否相等： True
(11463, 1)
长度是否相等： True
(978, 1)
长度是否相等： True
(1165, 1)
长度是否相等： True
(1976, 1)
长度是否相等： True
(37968, 1)
长度是否相等： True
(15010, 1)
长度是否相等： True
(967, 1)
长度是否相等： True
(128, 1)
长度是否相等： True


In [23]:
print(symptom)
print("疾病数：",len(sym["Mesh_sym_name"]))
sym["Mesh_sym_name"].iloc[380:390]  #循环结束到405，但是只有385个文件

Hirsutism
疾病数： 405


380    Purpura, Thrombotic Thrombocytopenic
381        Waterhouse-Friderichsen Syndrome
382                        Striae Distensae
383                              Sleepiness
384                  Travel-Related Illness
385                        Jet Lag Syndrome
386               Urological Manifestations
387                          Hypercalciuria
388            Lower Urinary Tract Symptoms
389                                 Dysuria
Name: Mesh_sym_name, dtype: object

In [69]:
"""Save the number of relevant pmids for each symptom"""
# pd.DataFrame(lenthDict,index=["count"]).to_csv("sym_PMID_count.csv")

### test

In [95]:
result.head()

,&term=Breast+Neoplasms[MAJR]
0,33378816
1,33378815
2,33378814
3,33378805
4,33378804


In [50]:
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&mindate=1966&maxdate=2020/12&retmode=json&usehistory=y"
query_term = "&term=Breast+Neoplasms[MAJR]"

url = base_url+query_term
#post the esearch URL
esearch_output = get_page(url)
print(esearch_output)

{'header': {'type': 'esearch', 'version': '0.3'}, 'esearchresult': {'count': '251838', 'retmax': '20', 'retstart': '0', 'querykey': '1', 'webenv': 'MCID_60d9870e09e62b09b274d24d', 'idlist': ['33378816', '33378815', '33378814', '33378805', '33378804', '33378803', '33378802', '33378801', '33378800', '33378415', '33378066', '33377172', '33377156', '33376736', '33376321', '33375141', '33375067', '33374849', '33374289', '33374116'], 'translationset': [{'from': 'Breast Neoplasms[MAJR]', 'to': '"breast neoplasms"[MeSH Major Topic]'}], 'translationstack': [{'term': '"breast neoplasms"[MeSH Major Topic]', 'field': 'MeSH Major Topic', 'count': '261794', 'explode': 'Y'}, {'term': '1966[EDAT]', 'field': 'EDAT', 'count': '0', 'explode': 'N'}, {'term': '2020/12[EDAT]', 'field': 'EDAT', 'count': '0', 'explode': 'N'}, 'RANGE', 'AND'], 'querytranslation': '"breast neoplasms"[MeSH Major Topic] AND 1966[EDAT] : 2020/12[EDAT]'}}


In [65]:
efetch_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&WebEnv=MCID_60d98bf865ea1328b61c1265&query_key=1&retstart=0&retmax=100000&rettype=uilist&retmode=text'

In [73]:
efetch_out.text #return type: str

'33378816\n33378815\n33378814\n33378805\n33378804\n33378803\n33378802\n33378801\n33378800\n33378415\n33378066\n33377172\n33377156\n33376736\n33376321\n33375141\n33375067\n33374849\n33374289\n33374116\n33373798\n33373363\n33372907\n33372855\n33371906\n33371679\n33371634\n33371333\n33371069\n33370954\n33370641\n33370310\n33370069\n33370046\n33370043\n33369357\n33368734\n33368563\n33367836\n33367678\n33367520\n33363371\n33361170\n33360931\n33360926\n33360919\n33360419\n33360155\n33360004\n33359886\n33359773\n33359275\n33359270\n33359260\n33359245\n33359199\n33358549\n33358209\n33358012\n33357918\n33357859\n33357805\n33357420\n33357419\n33357231\n33356943\n33356056\n33355509\n33355355\n33353943\n33353933\n33353392\n33353120\n33352945\n33352898\n33352842\n33352766\n33352739\n33352510\n33352497\n33352494\n33352145\n33351735\n33351674\n33351505\n33351008\n33350890\n33350778\n33350734\n33350732\n33350211\n33350210\n33349534\n33348862\n33348619\n33348436\n33348244\n33347821\n33347475\n33347397\

In [90]:
count

251838